In [1]:
#https://towardsdatascience.com/can-bots-tell-you-stories-357a77bef4c9
#https://github.com/jsvine/markovify
import markovify
import spacy
import numpy as np

## Data loading & cleaning

In [2]:
import io
f1 = "clean_zenmindbeginnermind.txt"
f2 = "clean_Branching-Streams.txt"
f3 = "clean_not_always_so.txt"
f4 = "Shunryu Suzuki Roshi lectures.txt"
# raw_text_zbm = io.open(f1).read()
# raw_text_bs = io.open(f2).read()
# raw_text_nas = io.open(f3).read()
# raw_text_lec = io.open(f4).read()
raw_text_zbm = open(f1,encoding='utf8', errors='ignore').read()
raw_text_bs = open(f2,encoding='utf8', errors='ignore').read()
raw_text_nas = open(f3,encoding='utf8', errors='ignore').read()
raw_text_lec = open(f4,encoding='utf8', errors='ignore').read()

In [3]:
raw_text = raw_text_zbm + raw_text_bs + raw_text_nas + raw_text_lec

In [5]:
import re
raw_text2 = raw_text.replace('- ', '') #join hyphenated words
raw_text2 = re.sub("[^ a-zA-Z.!\',:]", '', raw_text2) #remove other weird characters, remove numbers due to page no.
raw_text2 = re.sub(r'[A-Z]+(?![a-z ])', '', raw_text2) #?![... ]Matches if ... doesn’t match next. This is a negative lookahead assertion. For example, Isaac (?!Asimov) will match 'Isaac ' only if it’s not followed by 'Asimov'.
raw_text2 = re.sub(r'wiu', 'will', raw_text2)
raw_text2 = re.sub(r'wiU', 'will', raw_text2)
raw_text2 = re.sub(r'actuaUy', 'actually', raw_text2)
raw_text2 = re.sub(r'ActuaUy', 'Actually', raw_text2)
raw_text2 = re.sub(r'conunon', 'common', raw_text2)
raw_text2 = re.sub(r'groimd', 'ground', raw_text2)
raw_text2 = re.sub(r'diculties', 'difficulties', raw_text2)
raw_text2 = re.sub(r'diculty', 'difficulty', raw_text2)
raw_text2 = re.sub(r'dierence', 'difference', raw_text2)
raw_text2 = re.sub(r'socalled', 'so called', raw_text2)
raw_text2 = re.sub(r'fitSt', 'first', raw_text2)
raw_text2 = re.sub(r'pr.ctice', 'practice', raw_text2)
raw_text2 = re.sub(r'pmctice', 'practice', raw_text2)
raw_text2 = re.sub(r'caUigraphy', 'calligraphy', raw_text2)
raw_text2 = re.sub(r'eort', 'effort', raw_text2)
raw_text2 = re.sub(r'siUy', 'silly', raw_text2)
raw_text2 = re.sub(r'UsuaUy', 'Usually', raw_text2)
raw_text2 = re.sub(r'Ihere', 'there', raw_text2)
raw_text2 = re.sub(r'exacdy', 'exactly', raw_text2)
raw_text2 = re.sub(r'suering', 'suffering', raw_text2)
raw_text2 = re.sub(r'suer', 'suffer', raw_text2)
raw_text2 = re.sub(r'whcn', 'when', raw_text2)
raw_text2 = re.sub(r'nOlice', 'realise', raw_text2)
raw_text2 = re.sub(r'eventuaUy', 'eventually', raw_text2)
raw_text2 = re.sub(r'refleclive', 'reflective', raw_text2)
raw_text2 = re.sub(r'attenlive', 'attentive', raw_text2)
raw_text2 = re.sub(r'Ihroughoul', 'throughout', raw_text2)
raw_text2 = re.sub(r'olhers', 'others', raw_text2)
raw_text2 = re.sub(r'IlNn.DunnNnyou', 'you', raw_text2)
raw_text2 = re.sub(r'yearold', 'year old', raw_text2)
raw_text2 = re.sub(r'ofhow', 'of how', raw_text2)
raw_text2 = re.sub(r'oflife', 'of life', raw_text2)
raw_text2 = re.sub(r'ofI', 'of I', raw_text2)
raw_text2 = re.sub(r'everyoneevery', 'everyone every', raw_text2)
raw_text2 = re.sub(r'beingcan', 'being can', raw_text2)    
raw_text2 = re.sub(r'Butphilosophical', 'But philosophical', raw_text2)
raw_text2 = re.sub(r'[NEBSDPOTURG]+(?![a-z])', '', raw_text2) #remove indiviudal capital alphabets except I
# raw_text2 = re.sub(r'Kyoto', '', raw_text2) #remove kyoto sticking out
# raw_text2 = re.sub(r'Talk Given to a Visiting Classd i s c u s s i o ', '', raw_text2) #remove kyoto sticking out
raw_text2 = re.sub(r'\s+', ' ', raw_text2) #remove multiple spaces between strings and lower casing
raw_text2
#cannot remove '

"The following tribute from Trudy to her teacher describes very well the relationship between Zen teacher and Zen student: A roshi is a person who has actualized that perfect freedom which is the potentiality for all human beings. He exists freely in the fullness of his whole being. The flow of his consciousness is not the fixed repetitive patterns of our usual selfcentered consciousness, but rather arises spontaneously and naturally from the actual circumstances of the present. The results of this in terms of the quality of his life are extraordinary buoyancy, vigor, straightforwardness, simplicity, humility, serenity, joyousness, uncanny perspicacity and unfathomable compassion. His whole being testifies to what it means to live in the reality of the present. Without anything said or done, just the impact of meeting a personality so developed can be enough to change another's whole way of life. But in the end it is not the extraordinariness of the teacher which perplexes, intrigues, 

### Top zen words to recommend to user

In [8]:
import nltk
from nltk.tokenize import TweetTokenizer, sent_tokenize
import collections
from six.moves import cPickle

In [16]:
len(sentences)

2049

In [9]:
tokenizer_words = TweetTokenizer()
sentences = nltk.sent_tokenize(raw_text2) # get sentences
#get top words to recommend for user input
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
cvt = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
X_all = TfidfTransformer().fit_transform(cvt.fit_transform(sentences))
columns  =  np.array(cvt.get_feature_names())  

def get_freq_words(sparse_counts, columns):
    # X_all is a sparse matrix, so sum() returns a 'matrix' datatype ...
    #   which we then convert into a 1-D ndarray for sorting
    word_counts = np.asarray(X_all.sum(axis=0)).reshape(-1)
    # argsort() returns smallest first, so we reverse the result
    largest_count_indices = word_counts.argsort()[::-1]
    # pretty-print the results! Remember to always ask whether they make sense ...
    freq_words = pd.Series(word_counts[largest_count_indices], 
                           index=columns[largest_count_indices])
    return freq_words
freq_words = get_freq_words(X_all, columns)
freq_words[:20]

practice         297.593767
way              216.416322
mind             167.466866
zazen            160.889462
just             154.959317
buddha           153.907054
say              146.587681
true             127.230578
good             126.078918
understand       111.930319
think            111.867408
know             108.836075
understanding    105.419923
things           104.815058
enlightenment     98.192618
life              97.169933
zen               91.971603
actually          89.994304
nature            88.856819
right             80.991854
dtype: float64

In [8]:
freq_words[:50].index

Index(['practice', 'way', 'mind', 'zazen', 'just', 'buddha', 'say', 'true',
       'good', 'understand', 'think', 'know', 'understanding', 'things',
       'enlightenment', 'life', 'zen', 'actually', 'nature', 'right',
       'precepts', 'like', 'time', 'point', 'idea', 'teaching', 'kind',
       'means', 'people', 'feel', 'dont', 'big', 'sit', 'activity', 'doing',
       'important', 'thing', 'moment', 'bad', 'experience', 'buddhism',
       'words', 'mean', 'make', 'truth', 'effort', 'accept', 'thinking', 'don',
       'said'],
      dtype='object')

In [9]:
#get top words to recommend for user input
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
cvt = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
X_all = TfidfTransformer().fit_transform(cvt.fit_transform(sentences))
columns  =  np.array(cvt.get_feature_names())  

def get_freq_words(sparse_counts, columns):
    # X_all is a sparse matrix, so sum() returns a 'matrix' datatype ...
    #   which we then convert into a 1-D ndarray for sorting
    word_counts = np.asarray(X_all.sum(axis=0)).reshape(-1)
    # argsort() returns smallest first, so we reverse the result
    largest_count_indices = word_counts.argsort()[::-1]
    # pretty-print the results! Remember to always ask whether they make sense ...
    freq_words = pd.Series(word_counts[largest_count_indices], 
                           index=columns[largest_count_indices])
    return freq_words
freq_words = get_freq_words(X_all, columns)
freq_words[:30]
# import matplotlib.pyplot as plt
# import seaborn as sns
# plot = pd.DataFrame(freq_words[:30])
# fig, ax = plt.subplots(figsize=(11,6))
# g = sns.barplot(plot.index, plot[0])
# g.set_xticklabels(g.get_xticklabels(), rotation=45)
# plt;

practice         297.593767
way              216.416322
mind             167.466866
zazen            160.889462
just             154.959317
buddha           153.907054
say              146.587681
true             127.230578
good             126.078918
understand       111.930319
think            111.867408
know             108.836075
understanding    105.419923
things           104.815058
enlightenment     98.192618
life              97.169933
zen               91.971603
actually          89.994304
nature            88.856819
right             80.991854
precepts          79.612526
like              79.041138
time              76.934173
point             73.054250
idea              72.330146
teaching          70.352806
kind              69.669778
means             66.337236
people            64.721627
feel              64.318534
dtype: float64

## Markov

In [10]:
text_model = markovify.Text(raw_text2)

In [11]:
from autocorrect import spell
sent1 = text_model.make_sentence()
spellcheck = []
for word in sent1.split():
    spellcheck.append(spell(word))
' '.join(spellcheck)

'But when your mind when you just do it without having any rules and act in your practice'

In [12]:
sent1

'But when your mind when you just do it without having any rules and act in your practice.'

In [13]:
for i in range(5):
    print(text_model.make_sentence())

If the final unit, gokumi.
They liked talking about this way we will attain enlightenment.
Psychology will te you, I am at Tassajara: it is Truth itself.
So even though you are an absolute power in a special experience and think, This is the warning signal.
So Yakusan paid his respects to something we cannot survive.


In [14]:
for i in range(3):
    print(text_model.make_short_sentence(140))

Do not try to stop your emotional and thinking activity within.
And there is no other person, but for someone ! According to Dogen, we are alive, we are doing.
Although there are sixteen precepts, and we should not be so happy.


In our everyday problems.
This mind will be someone else.
A womans way may be mad at the same time dangerous.

In [15]:
# Recreate the model using 3 words state performs better
three_model = markovify.Text(raw_text2,state_size=3)
for i in range(5):
    print(three_model.make_sentence())

Baso said, I want to know what an important practice it is difficult to feci that vay because we are not bothered.
But actually we are not.
Of course this is not true observation of the precepts.
At least we must have confidence is not something for me to do this or not to hear the meaning behind the words.
Whether you have difficulties in your practice thatEis the spirit for you to sit.


In [16]:
input_word = input()
print (text_model.make_sentence_with_start(input_word, strict=False))
print (three_model.make_sentence_with_start(input_word, strict=False))


None
None


In [17]:
zen_json = three_model.to_json()
import json
with open('zen.json', 'w') as outfile:
    json.dump(zen_json, outfile)

In [18]:
def input_generator(input_word):
    input_word = input_word.lower()
    if three_model.make_sentence_with_start(input_word, strict=False) is None:
        return three_model.make_sentence_with_start(input_word.capitalize(), strict=False)
    else:
        return three_model.make_sentence_with_start(input_word, strict=False)

In [19]:
input_word = input()

input_generator(input_word)

hello


In [76]:
# #Build the second model from another author/book.
# ebook_b = clean_book('data/ebook2.txt')
# text_model_b = markovify.Text(ebook_b)
# for i in range(5):
#     print(text_model_b.make_sentence())

# #Combine the models into a single one
# both_models = markovify.combine([text_model_a,text_model_b])
# for i in range(5):
#     print(both_models.make_sentence())

### Markov actual breakdown

In [78]:
#Markov actual breakdown
#https://towardsdatascience.com/simulating-text-with-markov-chains-in-python-1a27e6d13fc6
corpus = raw_text2.split() #split into individual words & punctuations
def make_pairs(corpus): #to pair the words such that one word leads to the next
    for i in range(len(corpus)-1):
        yield (corpus[i], corpus[i+1])
pairs = make_pairs(corpus)
word_dict = {}
for word_1, word_2 in pairs: #put word pairs into dictionary to be called
    if word_1 in word_dict.keys():
        word_dict[word_1].append(word_2)
    else:
        word_dict[word_1] = [word_2]
        
first_word = np.random.choice(corpus) #choose random word from corpus
while first_word.islower(): #keep choosing till we get a Capitalised first word
    first_word = np.random.choice(corpus)
chain = ['A']
n_words = 50
for i in range(n_words):
    chain.append(np.random.choice(word_dict[chain[-1]])) #get the next word from dictionary in a random choice for range of 50
' '.join(chain)

'A Zen is significant. Because of zazen. And it all. Something that time, many do not matter of our precepts and that smallest extent. Just to express fully appreciating our precepts. So we believe in everyday life remains after taking over my opinion on page of being, which comes out just'

## Markov with Part of Speech

In [18]:
#https://towardsdatascience.com/simulating-text-with-markov-chains-in-python-1a27e6d13fc6
nlp = spacy.load("en")

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [47]:
comments_generator_POSified = POSifiedText(raw_text2, state_size = 3)

In [54]:
for i in range(5):
    print (comments_generator_POSified.make_sentence())

When you say this is the joy of eternal life .
That is the world of emptiness , we can understand our friends ways eventually .
In light you feel good just because you feel bad .
Buddha taught us how to act by sitting , and this is just a tentative form and color , and that is how to be a sincere disciple of the Sixth Ancestor in dierent versions , and each posture is Buddhas posture .
The precepts also should be observed without any idea of calamity or disaster , but even so I have to change my practice again to recite the sutra with a deep cairn feeling .


## Mixing physics with zen

In [12]:
f5 = "Reality-is-Not-What-It-Seems-The-Journey-to-Quantum-Gravity.txt"
raw_text_phy = open(f5,encoding='utf8', errors='ignore').read()

In [13]:
raw_text_phy2 = re.sub("[^ a-zA-Z.!\',:]", '', raw_text_phy) #remove other weird characters, remove numbers due to page no.
raw_text_phy2 = re.sub(r'[A-Z]+(?![a-z ])', '', raw_text_phy2)
raw_text_phy2 = re.sub(r'\s+', ' ', raw_text_phy2) #remove multiple spaces between strings and lower casing

In [14]:
raw_text_phy2

'During my entire research life, friends and curious people have asked me to explain what was going on in quantum gravity research. How was it possible to study new ways of thinking about space and time Over and over again I have been asked to write a popular account of this research. While books on cosmology or string theory abound, a book describing the research on the quantum nature of space and time, and on loop quantum gravity in particular, did not yet exist. I have long hesitated, because I wanted to concentrate on research. Some years ago, after completing my technical book on the subject, I felt that the collective work of many scientists had moved the topic to a stage mature enough for a popular book. The landscape we are exploring is enchanting: why keep it hiddenBut I still delayed the project, because I could not see the book in my head. How to explain a world without space and time One night in , during a long solitary drive from Italy to France, I realized that the only 

### Top physic words to recommend to user

In [15]:
#get top words to recommend for user input
tokenizer_words = TweetTokenizer()
sentences = nltk.sent_tokenize(raw_text_phy2) # get sentences
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
cvt = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
X_all = TfidfTransformer().fit_transform(cvt.fit_transform(sentences))
columns  =  np.array(cvt.get_feature_names())  

def get_freq_words(sparse_counts, columns):
    # X_all is a sparse matrix, so sum() returns a 'matrix' datatype ...
    #   which we then convert into a 1-D ndarray for sorting
    word_counts = np.asarray(X_all.sum(axis=0)).reshape(-1)
    # argsort() returns smallest first, so we reverse the result
    largest_count_indices = word_counts.argsort()[::-1]
    # pretty-print the results! Remember to always ask whether they make sense ...
    freq_words = pd.Series(word_counts[largest_count_indices], 
                           index=columns[largest_count_indices])
    return freq_words
freq_words = get_freq_words(X_all, columns)
freq_words[:20]
# import matplotlib.pyplot as plt
# import seaborn as sns
# plot = pd.DataFrame(freq_words[:30])
# fig, ax = plt.subplots(figsize=(11,6))
# g = sns.barplot(plot.index, plot[0])
# g.set_xticklabels(g.get_xticklabels(), rotation=45)
# plt;

space          45.706560
quantum        43.859141
time           40.619609
world          36.108474
theory         29.607166
figure         25.008309
universe       24.513984
gravity        23.755554
einstein       22.454986
mechanics      21.179752
light          20.530965
information    20.397645
field          19.726303
physics        19.507092
equations      19.439791
atoms          19.049262
way            18.858805
like           18.681235
things         18.658484
does           18.157500
dtype: float64

In [24]:
phy_model = markovify.Text(raw_text_phy2)
for i in range(5):
    print(phy_model.make_sentence())

It is at the list of names, each one of the Romans.The power of the microscopic randomness cancels out on average, leaving only fluctuations too minute for us to do something better than Einstein.
Yet we develop our conceptual schema for understanding the physics of the eighteenth century consisted of little other than reinforce the understanding of the atomic level.
Today, the inadequacy of our ignorance in the night sky.
But there is no consensus on how to interpret this equation, to know what they signify we dont consider them to improve its definition and measure length in multiples of Plancks length.
It should be different from matter.


In [25]:
phy_three_model = markovify.Text(raw_text_phy2,state_size=3)
for i in range(5):
    print(phy_three_model.make_sentence())

The dialogue continued for years, by way of which Western thought reconstructed itself, and nothing by Democritus.
The gravitational field is space figure ..But, unlike Newtons space, which is modified by electric and magnetic phenomena, to design antennae, radios, electric engines and computers.
There is one of the two balls move away in different directions.
Faraday and Maxwells fields, in turn, are made up of quantum fields which manifest themselves in the form of grains of finite size, infinitely small steps do not exist.
He refused to accept the idea that this was a good way to proceed, and clarified and developed this schema.


In [26]:
model_a = markovify.Text(raw_text2, state_size=3)
model_b = markovify.Text(raw_text_phy2, state_size=3)

model_combo1 = markovify.combine([ model_a, model_b ], [1, 1 ])
model_combo2 = markovify.combine([ model_a, model_b ], [1.1, 1 ])

In [27]:
for i in range(5):
    print(model_combo1.make_sentence())

Check to see that his idea of space the disappearance of time at small scale the physics of the heavens.
Do you understand How you understand my lecture, how you practice zazen, just to be yourself.As you have some security in your feeling.
To find the oneness of tongue and taste.
This is what Matvei Brontejn had intuited in the s. The calculation of the quantum beginning of our universe.One of the most intelligent and farseeing of men but also the whole world.
We practice zazen to attain enlightenment, then you will understand everything.


In [28]:
input_word = input()
print (model_combo1.make_sentence_with_start(input_word, strict=False))
print (model_combo2.make_sentence_with_start(input_word, strict=False))

physics
physics of these solutions lies in the effort we make.
physics of elementary particles which describes almost all we see, with the exception of gravity, but for the things themselves, that is the true bow.


In [29]:
zen_phy_json = model_combo2.to_json()
import json
with open('zen_phy.json', 'w') as outfile:
    json.dump(zen_phy_json, outfile)

## Productionising

In [30]:
import markovify
import spacy
import numpy as np
import json
with open('zen_phy.json', 'r') as f:
    model_json = json.load(f)
three_model = markovify.Text.from_json(model_json)

In [31]:
input_word = input()
from autocorrect import spell
input_word2 = spell(input_word)
try:
    gen_1 = 'Yours: ' + three_model.make_sentence_with_start(input_word2, strict=False)
except:
    gen_1 = 'None'

gen_2 = 'Zen Master says: ' + three_model.make_sentence() + ' '+ three_model.make_sentence()
gen_3 = gen_1 + '\n' + gen_2
print (gen_3)

universe
Yours: universe is fixed and immutable, but his equations tell him that this is just one side a Itllllbtlllktlll. someone who carries a board on his shoulder.
Zen Master says: This is the meaning of our life. When a tree stands up by itself, we call that understanding ri, or truth.


In [32]:
from autocorrect import spell
spellcheck = []
for word in gen_3.split():
    spellcheck.append(spell(word))
' '.join(spellcheck)

'Yours universe is fixed and immutable but his equations tell him that this is just one side a Itllllbtlllktlll. someone who carries a board on his shoulders Zen Master says This is the meaning of our life When a tree stands up by itself we call that understanding rid or truth'

In [20]:
for i in range(3):
    print(three_model.make_sentence())

When you listen to someone.
But when we are sitting together, you each sit in your own pure white screen, your practice will suffer.
But I dont want to live up in the morning we have to get up.


In [21]:
for i in range(3):
    print(three_model.make_short_sentence(140))

He wanted to meet a monk who wanted to be a sincere disciple of the Sixth Ancestor.
He asked himself whether there was a person who liked dragons very much.
If some extra idea comes, you should try to understand things once someone has thought them through.


In [22]:
input_word = input()
print (text_model.make_sentence_with_start(input_word, strict=False))
print (three_model.make_sentence_with_start(input_word, strict=False))
#use 3 model as default then 2 if None
#use spell check and upper first letter if input_word doesnt work

one
one you can say, This is true of human nature.
one at sea level and one in the mountains will find that, when they meet up again, the twin who lived in Alexandria in the first pages of On the Origin of Species.


### Spell correction

In [33]:
#spelling check without removing punctuations
from nltk.tokenize import TweetTokenizer
from autocorrect import spell
def cleaning(sentence):
    tokenizer_words = TweetTokenizer()
    spellcheck = []
    for word in tokenizer_words.tokenize(sentence):
        if word not in [',','!','.',':']:
            spellcheck.append(spell(word))
        else:
            spellcheck.append(word)
    corr_sent = ' '.join(spellcheck)
    return corr_sent

In [34]:
input_word = input()
from autocorrect import spell
input_word2 = spell(input_word)
try:
    gen_1 = cleaning(three_model.make_sentence_with_start(input_word2, strict=False))
    gen_1_clean = 'Yours: ' + gen_1
except:
    gen_1_clean = 'None'

gen_2_1 = cleaning(three_model.make_sentence())
gen_2_2 = cleaning(three_model.make_sentence())
gen_2_clean = 'Zen Master says: ' + gen_2_1
gen_3 = gen_1_clean + '\n' + gen_2_clean
print (gen_3)

universe
Yours: universe are not two , and when someone who is practicing our way is not to be disturbed , your effort will be refined more and more the problem will change into some subtle feeling .
Zen Master says: Then , whatever work you do , you will have perfect wisdom at that time .


## Generate for human/bot test

In [35]:
with open('zen.json', 'r') as f:
    model_json = json.load(f)
zen_model = markovify.Text.from_json(model_json)

In [37]:
for i in range(5):
    print(three_model.make_sentence())

So you should not have any idea of gain, and with the oneness of eye and form.
To live each moment, becoming one with everything, is the point you will understand your human nature, your intellectual faculty, and the truth present in your everyday life.
When you speak or act it will be digested in your tummy, you don't know what it is, but I still dont understand.
When you find it, you will lose control.
In some sense we should be able to is absurd but if I were your age I could agree with you now.


In [38]:
tokenizer_words = TweetTokenizer()
sentences = nltk.sent_tokenize(raw_text2) # get sentences
sentences[0]

'The following tribute from Trudy to her teacher describes very well the relationship between Zen teacher and Zen student: A roshi is a person who has actualized that perfect freedom which is the potentiality for all human beings.'

In [49]:
import random
for i in range(5):
    rn_no = random.randint(0, len(sentences) - 1)
    print (sentences[rn_no])

If you go to Japan by boat, it may take ten days, and by airplane, maybe ten hours.
To try to obtain freedom without being aware of the rules means nothing.
A practitioner doesnt become so involved with things either emotionally or intellectually, so it is easier to see things for what they are.Student: I have some difficulty listening to these talks.
If you keep practicing, you will naturally have this understanding and this feeling.
and you are Buddha.


In [50]:
import random
for i in range(5):
    rn_no = random.randint(0, len(sentences) - 1)
    print (sentences[rn_no])

The purpose of my talk is not to give you some intellectual understanding, but just to express my appreciation of our Zen practice.
To study ourselves is to forget ourselves.
But when you think, There are many earwigs here and they are harmful insects, so I have to kill this one, you understand things only in a dualistic way.
Usually we are very Today We May Be Very Happymuch attached to either the bright side or the dark side of things.
Sometimes we say Buddha nature.
